In [1]:
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

chromedriver = "/home/nate/Bench/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

import pandas as pd
import pickle

# scrape leafly

Before starting to scrape, always have to check the layout of the site and build a game plan.

For leafly, there's a page listing all of the strains of marijuana that have been reviewed, ordered by

number of reviews. It start's off just showing the top ~50. For the MVP I used beautiful soup to grab

these links, and later extended that code with a selenium script that clicked 'show me more' several

times. It's not more than a loop of clicks, with a url request, so I haven't included it.

In [15]:
link_list = pd.read_pickle('link_list.pkl')

Selenium scripts look way more intimidating than they really are. Basically just

enter developer mode 'CTRL+SHIFT+C', click on the object you're interested, and

copy x-path, boom success. For finicky sites (and internet connections) I prefer

to use try/except statements with sleep timers inside my functions, that way if

something goes wrong, it's easier to keep going.

In [3]:
def strain_scrape(strain_link, num_pages):
    
    '''
    given a link, scrape the reviews from each page for the given number of pages
    '''
    
    reviews = []
    
    for page in range(num_pages):
        time.sleep(5)
        driver.get('https://www.leafly.com' + strain_link + '/reviews?page={}&sort=date'.format(page))
        
        for review in range(1,9):
            time.sleep(1)
            try:
                driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                reviews.append(review_text)
                driver.execute_script("window.history.go(-1)")
            except:
                try:
                    time.sleep(5)
                    driver.navigate.refresh()
                    driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                    review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                    reviews.append(review_text)
                    driver.execute_script("window.history.go(-1)")
                except:
                    pass

    return reviews

Leafly has an initial popup that has one of two paths, and

then a second popup asking for an email address!

This next cell deals with that for us, and sets up the page for our scrape.

In [8]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.leafly.com')
try:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[1]/div[1]/fieldset/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[2]/button').click()
except:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/fieldset/div/span[1]/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/div/button').click()

time.sleep(60)
driver.find_element_by_xpath('/html/body/div[9]/div/div/div/div[2]/form/div/a').click()

# Data in Mongo

In [ ]:
from pymongo import MongoClient

In [ ]:
client = MongoClient(port = 12345)
reviews_db = client.reviewsdb

In [ ]:
collection = db.collection_names()
print(collection + ' contains: ' + db.collection.count() + ' different strains.'

Using our list of links, this cell actually does all of the scraping

by calling our earlier function. This whole process is very straight forward,

but it takes quite a bit of time.

Note that we have '13' in strain scrape. This is beacuse each page has 8 reviews,

and due to time constraints I decided to grab the 100 most recent reviews for 

as many strains as possible.

In [ ]:
for link in link_list:
    link_drop = []
    try:
        strain_reviews = strain_scrape(link, 13) # 13*8=104, some pages fail to load so it ranges
        link_drop.append(strain_reviews)
    except:
        pass
    db.collection.insert_one(link_drop)

## Bonus

I'm including this hacky bit because it was the only way for me to scrape

at one point.

First, the initialized list is moved into the function, so it gets cleared 

by each new strain. Pickle is in there too, so that each strain gets independently

appended into local storage (that way we can re-call it and throw it into Mongo).

Every project at Metis has had a moment like this...

In [9]:
for link in link_list[218:]:
    strain_list = []
    try:
        strain_reviews = strain_scrape(link, 13)
        strain_list.append(strain_reviews)
    except:
        pass
    check = pd.read_pickle('check_reviews.pkl')
    check = check + strain_list
    pickle_out = open('check_reviews.pkl', 'wb')
    pickle.dump(check, pickle_out)
    pickle_out.close()

In [23]:
# j=0
for i, c in enumerate(check):
    if len(c) == 0:
        del(check[i])
        del(link_list[i])
#         j+=1
# j

In [25]:
print(len(check), len(link_list))

201

In [27]:
pickle_out = open('link_list.pkl', 'wb')
pickle.dump(link_list, pickle_out)
pickle_out.close()